# 参考にしたブログ
https://www.st-hakky-blog.com/entry/2019/04/22/180619

# ※googlmapが表示されない方。
理由は、ブラウザが microsoft edgeだからです。右のリンクを参照して設定を変えましょう。
https://qiita.com/nsh1l/items/4bd7c6bbb3a2de49b91b

### c.NotebookApp.browser = u'\"C:\Program Files (x86)\Google\Chrome\Application\chrome.exe" %s --incognito'
### ↑↑シャープは消します

In [1]:
import math
from datetime import datetime, timedelta
import folium
import csv
import pandas as pd

In [2]:
#下準備
# 時刻を表示するときに、どういう文字列として表示するかを決めておく。細かい意味はstrftime等を参照
time_format = "%Y-%m-%d %H:%M:%S.%f"

# 度からラジアンに変換
def to_radian(degree):
    return float(degree) * math.pi / 180.0

# 緯度経度で与えられた2点間の距離を近似計算
def calc_distance(lat1, lng1, lat2, lng2):
    a = 6378137.000
    e2 = 0.00669437999019758
    mnum = 6335439.32729246
    lat1 = to_radian(lat1)
    lng1 = to_radian(lng1)
    lat2 = to_radian(lat2)
    lng2 = to_radian(lng2)
    my = (lat1 + lat2) / 2.0
    dy = lat1 - lat2
    dx = lng1 - lng2

    sin = math.sin(my)
    cos = math.cos(my)
    w = math.sqrt(1.0 - e2 * sin * sin)
    m = mnum / (w * w * w)
    n = a / w
    
    dym = dy * m
    dxncos = dx * n * cos
    
    return math.sqrt(dym * dym + dxncos * dxncos)

In [101]:
positions  = [(35.654155, 139.795887),#豊洲
              (35.663487, 139.783999),#月島
              (35.670273, 139.773506),#新富町
              (35.674199, 139.766994),#銀座一丁目
              (35.675062, 139.762466),#有楽町
              #(縦(経度), 横(緯度))
             ]


# ぐるなびとの連動
https://api.gnavi.co.jp/api/manual/

In [4]:
import requests
import time

In [102]:
Url = "https://api.gnavi.co.jp/PhotoSearchAPI/v3/"
shop_dic = pd.DataFrame()
#パラメータの設定
for tate, yoko in positions:
    params={}
    params["keyid"] = "!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!" #取得したアクセスキー
    params["latitude"] = float(tate)
    params["longitude"] = float(yoko)
    params["range"] = 2 #1:300m、2:500m、3:1000m、4:2000m、5:3000m
    params["photo_scene_id"] = 2 #2=宴会
    params["hit_per_page"] =50 #50 is max
    #リクエスト結果
    shop_dic = shop_dic.append(requests.get(Url, params).json(),ignore_index=True)
    time.sleep(1)

In [103]:
shops = pd.DataFrame()
dic_num = 0
times = 1
temp_rank = 0.0
temp_umaso = 0
temp_kome = ""
lastflag = False
for get in shop_dic["response"]:
    lastflag= False
    maxlen = len(shop_dic["response"][dic_num].keys())-5
    dic_num= dic_num + 1
    for i in range(maxlen+1):
        if i == maxlen:
            lastflag = True
        else:
            pass
    
        if lastflag == True:
            try:
                rank = float(get[str(i)]["photo"]["total_score"])
            except KeyError as e:
                rank = 2.0                
            try:
                umaso = float(get[str(i)]["photo"]["umaso_count"])
            except KeyError as e:
                umaso = 0
            
            temp_s   = pd.DataFrame([[get[str(i)]["photo"]["shop_id"],
                                      get[str(i)]["photo"]["shop_name"],
                                      get[str(i)]["photo"]["category"],
                                     (temp_rank + rank)/times,
                                     (temp_umaso + umaso)/times,
                                      temp_kome + get[str(i)]["photo"]["comment"] + "。"]])       
            shops = shops.append(temp_s)
            temp_rank = 0.0
            temp_umaso = 0
            temp_kome = ""        
            times = 1
        elif (get[str(i)]["photo"]["shop_name"] != get[str(i+1)]["photo"]["shop_name"]):
            try:
                rank = float(get[str(i)]["photo"]["total_score"])
            except KeyError as e:
                rank = 2.0
            try:
                umaso = float(get[str(i)]["photo"]["umaso_count"])
            except KeyError as e:
                umaso = 0

            
            temp_s   = pd.DataFrame([[get[str(i)]["photo"]["shop_id"],
                                      get[str(i)]["photo"]["shop_name"],
                                      get[str(i)]["photo"]["category"],
                                     (temp_rank + rank)/times,
                                     (temp_umaso + umaso)/times,
                                      temp_kome + get[str(i)]["photo"]["comment"] + "。"]])       
            shops = shops.append(temp_s)
            temp_rank = 0.0
            temp_umaso = 0
            temp_kome = ""
            times = 1
        else:
            try:
                rank = float(get[str(i)]["photo"]["total_score"])
            except KeyError as e:
                rank = 2.0
            try:
                umaso = float(get[str(i)]["photo"]["umaso_count"])
            except KeyError as e:
                umaso = 0
                
            temp_rank = temp_rank + rank
            temp_umaso = temp_umaso + rank
            temp_kome = temp_kome + get[str(i)]["photo"]["comment"] + "。"
            times     = times + 1

In [94]:
#shops[:]

In [104]:
shops_sel = shops.copy()

In [105]:
#平均評価が、2点以下は除外。
shops_sel = shops_sel[shops_sel[3]>2]

In [106]:
#それぞれの点で、同じ店を抽出することも多々ある。よって、重複削除
shops_sel = shops_sel.drop_duplicates() 

In [107]:
shops_sel.columns = ["id", "店名","店種類","評価","うまそ数","口コミ"]

In [108]:
shops_sel = shops_sel.sort_values("評価",ascending=False)
shops_sel

,id,店名,店種類,評価,うまそ数,口コミ
0,g084664,鶏料理専門店×個室 とりかく丸の内国際ビル店,個室 焼き鳥 有楽町,5.000000,0.000000,送別会に、飲み放題付きでお願いしました。\r\nお料理は、ボリュームたっぷりで、お味も大変美...
0,g341504,The Stay Gold GINZA貸切パーティー,銀座貸切パーティー会場,5.000000,0.000000,ステイゴールド銀座さんは、昨年12月17日に忘年会で利用させていただいたのですが、予想以上...
0,g725415,裏NO庭銀座店,銀座個室居酒屋和食宴会,4.500000,3.000000,舌触りがよくものすごく滑らかで表面のカリカリになったカラメルが病みつきになりました！\r\n...
0,e155300,ぴょんぴょん舎 GINZA UNA,美味しい焼肉と盛岡冷麺,4.500000,3.000000,特選肉をいくつか。\r\nさすが特選というだけあって、とろける美味しさでした。。
0,g714602,銀座ロビー,厳選食材の和食とお酒,4.250000,2.500000,めちゃクチャ大きな立派なエビ。きた瞬間見た目で美味しい。歯応えもぷりぷり申し分ない。頭まで食...
0,p458315,古民家個室居酒屋 尺山寸水有楽町店,有楽町・居酒屋・個室,4.166667,3.866667,友人たちと銀座をぶらつき、有楽町のほうへ歩いてたら見つけたお店。最近オープンしたみたいで店内...
0,ga1u203,すし 魚游（うおゆう）銀座店,銀座/完全個室/寿司,4.000000,1.000000,お刺身もりもり！えびはかなりの大物でした。コスパ、かなりいいです。。
0,gc0x101,で～びる沖縄,銀座 沖縄料理・ライブ,4.000000,4.125000,たまごとスパムというシンプルな料理。スパムがおいしいのはもちろん、付け合わせのサラダも美味し...
0,6283021,へたれ！？,居酒屋,4.000000,0.000000,お任せで色々盛ってもらいました(*´ω｀*)『２度付け厳禁』のソースはさっぱり系でどっぷり浸...
0,gf5n600,個室×和食銀座 本陣房～ほんじんぼう～,鮮魚・鴨・蕎麦・日本酒,4.000000,0.000000,宴会で利用させていただきました！【期間限定】として特別に宴会メニューに寒ブリの黄身天巻きを提...


# 口コミをネガポジ分析して、口コミを数値化する！

In [109]:
#口コミの数値を入れる列を追加。
shops_sel[6] = 0

In [110]:
#★★ Janome 'Hello 蛇の目'★★
from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.tokenfilter import *

import codecs

In [111]:
class CorpusElement:
    def __init__(self, text='', tokens=[], pn_scores=[]):
        self.text = text # テキスト本文
        self.tokens = tokens # 構文木解析されたトークンのリスト
        self.pn_scores = pn_scores # 感情極性値(後述)

# pn_ja.dicファイルから、単語をキー、極性値を値とする辞書を得る
def load_pn_dict():
    dic = {}
    
    with codecs.open('nega_poji.dic', 'r', 'UTF-8') as f:
        lines = f.readlines()
        
        for line in lines:
            # 各行は"良い:よい:形容詞:0.999995"
            columns = line.split(':')
            dic[columns[0]] = float(columns[3])
            
    return dic


# トークンリストから極性値リストを得る
def get_pn_scores(tokens, pn_dic):
    scores = []
    
    for surface in [t.surface for t in tokens if t.part_of_speech.split(',')[0] in ['形容詞']]:
        if surface in pn_dic:
            scores.append(pn_dic[surface])
        
    return scores

In [120]:
for i,texts in enumerate(shops_sel.iloc[:,5]):
    # CorpusElementのリスト
    naive_corpus = []
    naive_tokenizer = Tokenizer()
    tokens = naive_tokenizer.tokenize(texts)
    element = CorpusElement(texts, tokens)
    naive_corpus.append(element)   

    # 感情極性対応表のロード
    pn_dic = load_pn_dict()
    # 各文章の極性値リストを得る
    for element in naive_corpus:
        element.pn_scores = get_pn_scores(element.tokens, pn_dic)
    shops_sel.iloc[i,6] = sum(element.pn_scores)

1.9132259999999999
0.9835360000000001
1.9516179999999999
0
0.982723
2.154843
0
3.96544
0.999995
0.921866
0.987973
0
1.98272
-0.878433
0.99136
-0.999237
-1.288046
0.070708
1.98272
0
0
2.878019
1.98272
3.1191900000000006
1.972801


In [123]:
#人気の店、ベスト３を教える。
shops_sel[:]

,id,店名,店種類,評価,うまそ数,口コミ,6
0,g084664,鶏料理専門店×個室 とりかく丸の内国際ビル店,個室 焼き鳥 有楽町,5.000000,0.000000,送別会に、飲み放題付きでお願いしました。\r\nお料理は、ボリュームたっぷりで、お味も大変美...,1.913226
0,g341504,The Stay Gold GINZA貸切パーティー,銀座貸切パーティー会場,5.000000,0.000000,ステイゴールド銀座さんは、昨年12月17日に忘年会で利用させていただいたのですが、予想以上...,0.983536
0,g725415,裏NO庭銀座店,銀座個室居酒屋和食宴会,4.500000,3.000000,舌触りがよくものすごく滑らかで表面のカリカリになったカラメルが病みつきになりました！\r\n...,1.951618
0,e155300,ぴょんぴょん舎 GINZA UNA,美味しい焼肉と盛岡冷麺,4.500000,3.000000,特選肉をいくつか。\r\nさすが特選というだけあって、とろける美味しさでした。。,0.000000
0,g714602,銀座ロビー,厳選食材の和食とお酒,4.250000,2.500000,めちゃクチャ大きな立派なエビ。きた瞬間見た目で美味しい。歯応えもぷりぷり申し分ない。頭まで食...,0.982723
0,p458315,古民家個室居酒屋 尺山寸水有楽町店,有楽町・居酒屋・個室,4.166667,3.866667,友人たちと銀座をぶらつき、有楽町のほうへ歩いてたら見つけたお店。最近オープンしたみたいで店内...,2.154843
0,ga1u203,すし 魚游（うおゆう）銀座店,銀座/完全個室/寿司,4.000000,1.000000,お刺身もりもり！えびはかなりの大物でした。コスパ、かなりいいです。。,0.000000
0,gc0x101,で～びる沖縄,銀座 沖縄料理・ライブ,4.000000,4.125000,たまごとスパムというシンプルな料理。スパムがおいしいのはもちろん、付け合わせのサラダも美味し...,3.965440
0,6283021,へたれ！？,居酒屋,4.000000,0.000000,お任せで色々盛ってもらいました(*´ω｀*)『２度付け厳禁』のソースはさっぱり系でどっぷり浸...,0.999995
0,gf5n600,個室×和食銀座 本陣房～ほんじんぼう～,鮮魚・鴨・蕎麦・日本酒,4.000000,0.000000,宴会で利用させていただきました！【期間限定】として特別に宴会メニューに寒ブリの黄身天巻きを提...,0.921866


In [21]:
# マップを作成
shop_map = folium.Map()
shop_position = []
Url = "https://api.gnavi.co.jp/RestSearchAPI/v3/"
#パラメータの設定
for i in range(3):
    params={}
    params["keyid"] = "359554eb45ca190ed764c0a22ff80b82" #取得したアクセスキー
    params["id"] =shops_sel.iloc[i,0]
    #リクエスト結果
    best = requests.get(Url, params).json()
    folium.Marker(location=[float(best["rest"][0]["latitude"]), 
                        float(best["rest"][0]["longitude"]) ] ).add_to(shop_map)
    time.sleep(1)
shop_map